In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import tensorflow_datasets as tfds
import tensorflow as tf
from konlpy.tag import Okt
import time
from tqdm import tqdm
from random import sample

converter = Okt()

In [48]:
converter.pos("컴퓨터 개발자 구인합니다.".replace('비전','비젼'))

[('컴퓨터', 'Noun'),
 ('개발자', 'Noun'),
 ('구인', 'Noun'),
 ('합니다', 'Verb'),
 ('.', 'Punctuation')]

In [2]:
total_df = pd.read_csv('total_df.csv')
shuffle_df1 = total_df.sample(frac=1).reset_index(drop=True)
shuffle_df2 = total_df.sample(frac=1).reset_index(drop=True)
shuffle_df3 = total_df.sample(frac=1).reset_index(drop=True)

title = total_df['title']
labels = pd.read_csv('label.csv')

sector_dict = dict(zip(list(labels['label']),list(labels['index'])))

sector_label = list(total_df['sector'])
shuffle1_label = list(shuffle_df1['sector'])
shuffle2_label = list(shuffle_df2['sector'])
shuffle3_label = list(shuffle_df3['sector'])

In [3]:
sentences = []
label = []
for index,title in tqdm(enumerate(total_df['title'])) : #토큰
    okt_title = converter.pos(title)
    sentences.append(' '.join([tup[0].upper() for tup in okt_title if tup[1] == 'Noun' or tup[1] == 'Alpha']))

for index,title in tqdm(enumerate(shuffle_df1['title'])) : #전체 문장
    okt_title = converter.pos(title)
    token = [tup[0].upper() for tup in okt_title if tup[1] == 'Noun' or tup[1] == 'Alpha']
    sentences.append(' '.join(sample(token,len(token))))
    
for index,title in tqdm(enumerate(shuffle_df2['title'])) : #전체 문장 섞음
    okt_title = converter.pos(title)
    token = [tup[0].upper() for tup in okt_title if tup[1] == 'Noun' or tup[1] == 'Alpha']
    sentences.append(' '.join(sample(token,len(token))))
sentences

3481it [00:04, 786.36it/s] 
3481it [00:01, 2369.75it/s]
3481it [00:01, 2355.44it/s]


['NODE JS BACKEND 개발자',
 '단비 NODE JS REACT TYPESCRIPT 개발자',
 'JAVA 개발 비즈니스 플랫폼',
 'COUPANG PAY PRINCIPAL TECHNICAL PROGRAM MANAGEMENT',
 '핀 테크 파운트 FULL STACK ENGINEER',
 '핀 테크 파운트 프론트엔드 개발자',
 '백엔드 서비스 개발',
 '다짐 백엔드 개발자 이상',
 'PHP MYSQL JAVA 정규 경력 개발자 모집',
 '프론트엔드 개발자 신입 채용',
 '크립 토네이도 서버 개발자',
 '블록 체인 서비스 백엔드 개발자',
 '더 리움 클레이튼 기반 플랫폼 개발자',
 '어드민 풀 스택 개발자',
 'QA 담당자',
 '코리아 웹개발자 앱 개발자 경력 채용',
 '인슈 테크 스타트업 주 웰그램 개발자',
 '프리 윌린 프론트엔드 개발자 주니어',
 'QA QA ENGINEER 매 플랫',
 '프리 윌린 프론트엔드 개발자 시니어',
 '개발 IOS 개발자 B C',
 '프리 윌린 백엔드 개발자 시니어',
 'QA 엔지니어',
 'IOS 개발자',
 'BACKEND DEVELOPER',
 'FRONTEND ENGINEER',
 '사업 TYPESCRIPT BACKEND 엔지니어',
 'FRONT END DEVELOPER ANGULAR',
 '개발 팀 시니어 FULL STACK 개발자 정규직 경력 모집',
 '개발 팀 시니어 FRONT END 개발자 정규직 경력 모집',
 '개발 팀 사내 프로젝트 서버 개발 정규직 경력 모집',
 '프론트엔드 엔지니어 시니어',
 '데이터 엔지니어',
 '백엔드 엔지니어 시니어',
 'IOS 엔지니어',
 '안드로이드 엔지니어',
 '임 포트 개발 기획 동료 지원',
 '차이 카드 페이 QA ENGINEER',
 '차이 카드 페이 DEVOPS ENGINEER',
 '임 포트 SENIOR FRONT END ENGINEER',
 '임 포트 BACK END ENGINEER',
 '임 포트 IOS EN

In [4]:
len(sentences)

10443

In [5]:
sector_label = sector_label+shuffle1_label+shuffle2_label

In [6]:
len(sector_label)

10443

In [7]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    sentences + sector_label, target_vocab_size=2**13)

In [8]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

print('시작 토큰 번호 :',START_TOKEN)
print('종료 토큰 번호 :',END_TOKEN)
print('단어 집합의 크기 :',VOCAB_SIZE)

시작 토큰 번호 : [2792]
종료 토큰 번호 : [2793]
단어 집합의 크기 : 2794


In [9]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 20

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
        # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)

        # 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

    return tokenized_inputs, tokenized_outputs

In [10]:
questions, answers = tokenize_and_filter(sentences, sector_label)

In [11]:
print('질문 데이터의 크기(shape) :', questions.shape)
print('답변 데이터의 크기(shape) :', answers.shape)

질문 데이터의 크기(shape) : (10443, 20)
답변 데이터의 크기(shape) : (10443, 20)


In [12]:
print(questions[0])
print(answers[0])

[2792  111   79   50    4 2793    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
[2792    3 2581    1 2793    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [13]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [14]:
print(answers[0]) # 기존 샘플
print(answers[:1][:, :-1]) # 마지막 패딩 토큰 제거하면서 길이가 39가 된다.
print(answers[:1][:, 1:]) # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. 길이는 역시 39가 된다.

[2792    3 2581    1 2793    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
[[2792    3 2581    1 2793    0    0    0    0    0    0    0    0    0
     0    0    0    0    0]]
[[   3 2581    1 2793    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0]]


In [15]:
from model.transformer import transformer,CustomSchedule,loss_function

tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

(1, 2794, 256)
(1, 2794, 256)
Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    1769472     inputs[0][0]                     
                                                          

In [17]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# def accuracy(y_true, y_pred):
#     # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
#     y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
#     return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=['accuracy'])

In [18]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
164/164 [==============================] - 24s 124ms/step - loss: 1.3103 - accuracy: 0.0231
Epoch 2/50
164/164 [==============================] - 21s 126ms/step - loss: 0.6234 - accuracy: 0.1035
Epoch 3/50
164/164 [==============================] - 21s 126ms/step - loss: 0.1844 - accuracy: 0.1510
Epoch 4/50
164/164 [==============================] - 20s 125ms/step - loss: 0.0585 - accuracy: 0.1743
Epoch 5/50
164/164 [==============================] - 20s 124ms/step - loss: 0.0240 - accuracy: 0.1790
Epoch 6/50
164/164 [==============================] - 20s 124ms/step - loss: 0.0122 - accuracy: 0.1827
Epoch 7/50
164/164 [==============================] - 21s 125ms/step - loss: 0.0074 - accuracy: 0.1822
Epoch 8/50
164/164 [==============================] - 20s 124ms/step - loss: 0.0056 - accuracy: 0.1831
Epoch 9/50
164/164 [==============================] - 21s 127ms/step - loss: 0.0047 - accuracy: 0.1835
Epoch 10/50
164/164 [==============================] - 21s 125ms/step - l

In [19]:
def evaluate(sentence):
    sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    output = tf.expand_dims(START_TOKEN, 0)

    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

        # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if tf.equal(predicted_id, END_TOKEN[0]):
              break

        # 마지막 시점의 예측 단어를 출력에 연결한다.
        # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0)

In [52]:
def predict(sentence):
    sentence = sentence.upper()
    okt_title = converter.pos(sentence)
    sentence = ' '.join([tup[0].upper() for tup in okt_title if tup[1] == 'Noun' or tup[1] == 'Alpha'])
    prediction = evaluate(sentence)

    predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

In [21]:
output = predict("데이터 프로덕트 매니저")

Input: 데이터 프로 덕트 매니저
Output: Project-manager


In [22]:
output = predict("클라이언트 개발자 (캐주얼 게임)")

Input: 클라이언트 개발자 캐 주얼 게임
Output: Game


In [23]:
output = predict("Tech Lead Manager")

Input: TECH LEAD MANAGER
Output: CTO


In [24]:
output = predict("머신러닝/데이터 분석가")

Input: 머신 러닝 데이터 분석
Output: Machine-learning


In [25]:
output = predict("풀스택 엔지니어(2년 이상)")

Input: 풀 스택 엔지니어 이상
Output: WEB/Full-stack


In [26]:
output = predict("C++ 개발자")

Input: C 개발자
Output: C#/C++/C


In [27]:
output = predict("Exchange Admin Console Developer")

Input: EXCHANGE ADMIN CONSOLE DEVELOPER
Output: C#/C++/C


In [28]:
output = predict("UX 디자인")

Input: UX 디자인
Output: Web-publisher


In [29]:
output = predict("커머스 앱 및 시스템 앱을 위한 React Native 개발자")

Input: 커머스 앱 및 시스템 앱 위 REACT NATIVE 개발자
Output: Mobile


In [53]:
output = predict("unity를 활용한 플랫폼 개발자")

Input: UNITY 를 활용 플랫폼 개발자
Output: Unity/AR/VR/3D


In [54]:
output = predict("back")

Input: BACK
Output: Back-end


In [66]:
predict("서버개발자(Frontend Engineer)")

Input: 서버 개발자 FRONTEND ENGINEER
Output: WEB/Full-stack


'WEB/Full-stack'

In [34]:
correct = 0
wrong = 0

for index,title in tqdm(enumerate(total_df['title'])) : 
    output = predict(title)
    if output == sector_label[index] :
        correct += 1
    else :
        wrong +=1
        
print(f'correct : {correct}, wrong : {wrong}')
print(f'real acc : {correct/3481}')

3481it [04:34, 12.67it/s]

correct : 3436, wrong : 45
real acc : 0.9870726802642918


In [35]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [49]:
# model_json = model.to_json()
# with open("transformer.json", "w") as json_file : 
#     json_file.write(model_json)
    
model.save("transformer.h5")
print("Saved model to disk")

NotImplementedError: Layer PositionalEncoding has arguments in `__init__` and therefore must override `get_config`.

In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [40]:
from tensorflow.keras.models import load_model

loaded_model = load_model('transformer.h5')
loaded_model.summary()

ValueError: No model found in config file.